In [3]:
import requests
import sqlite3

# URLからJSONデータを取得
url = "http://www.jma.go.jp/bosai/common/const/area.json"
response = requests.get(url)

# レスポンスのエンコーディングをUTF-8に設定
response.encoding = 'utf-8'

# JSONデータをPythonの辞書として解析
area_json = response.json()

# "centers" キー内のデータを取得
centers = area_json.get("centers", {})

# SQLiteデータベース接続
db_filename = 'area_info.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# テーブルの作成（もしテーブルが存在しない場合）
cursor.execute("""
CREATE TABLE IF NOT EXISTS area_info (
    id INTEGER PRIMARY KEY,
    area_code TEXT,
    area_name TEXT,
    en_name TEXT,
    office_name TEXT,
    children_code TEXT
)
""")

# idの初期化
id_counter = 1

# 各エリア情報をSQLiteデータベースに挿入
for area_code, area_info in centers.items():
    name = area_info.get("name", "N/A")
    enName = area_info.get("enName", "N/A")
    officeName = area_info.get("officeName", "N/A")
    children = area_info.get("children", [])

    # 親エリアの情報をSQLiteに書き込む
    if children:
        for region_code in children:
            # 子エリアIDを挿入
            cursor.execute("""
            INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (id_counter, area_code, name, enName, officeName, region_code))
            id_counter += 1
    else:
        # 子エリアがいない場合
        cursor.execute("""
        INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (id_counter, area_code, name, enName, officeName, 'N/A'))
        id_counter += 1

# コミットして変更を保存
conn.commit()

# データベース接続を閉じる
conn.close()

OperationalError: database is locked